# Análisis de Importación de Vehículos desde el Portal SAT

In [15]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
import csv

In [16]:
urlSat = "https://portal.sat.gob.gt/portal/alza-e-importacion-vehiculos/#1510763502681-dff4b62b-fd76"
baseUrl = "https://portal.sat.gob.gt"
rutaSalida = "./datos/enlaces-importacion-2024_2025.txt"
carpetaZips = "./datos/zips/"
carpetaDescomprimidos = "./datos/descomprimidos/"
archivoUnificado = "./datos/data_unificada.csv"


## Código para bajar los enlaces necesarios

Podemos en un futuro solo aplicar el codigo para descarga de descomprimir los archivos dado los enlaces sin hacer la búsqueda.

In [ ]:
def obtenerImportacionesZips(url):
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    patronZip = re.compile(r"\.zip$", re.I)
    patronImportacion = re.compile(r"/importacion-de-vehiculos/", re.I)

    enlaces = set()

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if (
            patronZip.search(href)
            and ("2024" in href or "2025" in href)
            and patronImportacion.search(href)
        ):
            enlaces.add(urljoin(baseUrl, href))

    return sorted(enlaces)

In [ ]:
def guardarEnlacesImportacion(forzar=False):
    if os.path.exists(rutaSalida) and not forzar:
        respuesta = input(f"El archivo '{rutaSalida}' ya existe. ¿Deseas rehacer el proceso? (s/n): ").strip().lower()
        if respuesta != "s":
            print("Proceso cancelado.")
            return

    print("Obteniendo enlaces...")
    enlaces = obtenerImportacionesZips(urlSat)

    os.makedirs(os.path.dirname(rutaSalida), exist_ok=True)

    with open(rutaSalida, "w", encoding="utf-8") as f:
        f.write("\n".join(enlaces))

    print(f"{len(enlaces)} enlaces guardados en {rutaSalida}")

### Hacer WebScrapping

Se puede cambiar el valor **forzar** a true para hacer la descarga automatica de los enlaces (aún si existe el archivo de enlaces en `./data/enlaces-importacion-2024_2025.txt`)

In [ ]:
guardarEnlacesImportacion(forzar=False)

## Código para descomprimir los archivos Zip

In [ ]:
def descargarYDescomprimirZips(ejecutar=True):
    """Descarga y descomprime los ZIP listados en rutaSalida si ejecutar=True."""
    if not ejecutar:
        print("Descarga y descompresión omitidas (ejecutar=False).")
        return

    if not os.path.exists(rutaSalida):
        print(f"No existe el archivo de enlaces: {rutaSalida}")
        return

    os.makedirs(carpetaZips, exist_ok=True)
    os.makedirs(carpetaDescomprimidos, exist_ok=True)

    with open(rutaSalida, "r", encoding="utf-8") as f:
        enlaces = [line.strip() for line in f if line.strip()]

    for enlace in enlaces:
        nombreZip = os.path.basename(enlace)
        rutaZip = os.path.join(carpetaZips, nombreZip)

        if not os.path.exists(rutaZip):
            print(f"Descargando: {nombreZip}...")
            try:
                r = requests.get(enlace, timeout=30)
                r.raise_for_status()
                with open(rutaZip, "wb") as fzip:
                    fzip.write(r.content)
                print(f"Descargado: {rutaZip}")
            except Exception as e:
                print(f"Error al descargar {enlace}: {e}")
                continue
        else:
            print(f"Ya existe: {rutaZip}")

        try:
            with zipfile.ZipFile(rutaZip, 'r') as zip_ref:
                zip_ref.extractall(carpetaDescomprimidos)
                print(f"Descomprimido en: {carpetaDescomprimidos}")
        except zipfile.BadZipFile:
            print(f"Archivo ZIP corrupto o inválido: {rutaZip}")

### Descomprimir Zips

Se puede modificar el campo `ejecutar` a true o false para ejecutar el proceso de descarga y descomprimir archivos.

In [ ]:
descargarYDescomprimirZips(ejecutar=False)

## Unificar archivos descomprimidos

In [19]:
def unificarArchivosTxt(ejecutar=True):
    if not ejecutar:
        print("Unificación de archivos omitida (ejecutar=False).")
        return

    archivosTxt = [
        f for f in os.listdir(carpetaDescomprimidos)
        if f.endswith(".txt")
    ]

    if not archivosTxt:
        print("No hay archivos .txt en la carpeta descomprimidos.")
        return

    registros = []
    encabezado = None

    for archivo in archivosTxt:
        ruta = os.path.join(carpetaDescomprimidos, archivo)

        # Intentar abrir en utf-8, luego latin-1 si falla
        try:
            with open(ruta, "r", encoding="utf-8") as f:
                lineas = [line.strip() for line in f if line.strip()]
        except UnicodeDecodeError:
            with open(ruta, "r", encoding="latin-1") as f:
                lineas = [line.strip() for line in f if line.strip()]

        if not lineas:
            continue

        encabezadoArchivo = lineas[0]
        if encabezado is None:
            encabezado = encabezadoArchivo.split("|")
        elif encabezadoArchivo != "|".join(encabezado):
            print(f"Encabezado inconsistente en: {archivo}, se omitirá.")
            continue

        for linea in lineas[1:]:
            registros.append(linea.split("|"))

    # Guardar CSV en UTF-8
    os.makedirs(os.path.dirname(archivoUnificado), exist_ok=True)
    with open(archivoUnificado, "w", newline="", encoding="utf-8") as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow(encabezado)
        writer.writerows(registros)

    print(f"{len(registros)} registros guardados en {archivoUnificado}")

### Unificar archivos

Se puede modificar el campo `ejecutar` a true o false para ejecutar el proceso unificación de archivos.

In [20]:
unificarArchivosTxt(ejecutar=True)

962721 registros guardados en ./datos/data_unificada.csv


## Exploración de datos

### Análisis para el año 2024

#### ¿Cuántos **vehículos livianos** de **cada tipo** se importaron en 2024?

#### ¿Cuál es la **distribución de modelos** de **carros**, **pickups** y **SUV** importados en ese año?

#### ¿Cuál es el **tipo de vehículo** que más se importó durante el 2024?

#### ¿Cuáles fueron los **meses con mayor importación** de vehículos livianos?

### Comparativo 2024 vs 2025

#### ¿Cómo vamos con la importación de **cada tipo de vehículo** en los primeros meses de 2025 comparado con los mismos meses de 2024?